In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

%matplotlib inline

In [2]:
ls data

contest1/  contest2/


In [5]:
train = pd.read_csv('./data/contest1/train.csv', index_col=0)
train[:3]

,label,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
id,,,,,,,,,,,,,,,,,,,,,
1,1,7,0,3,0,2,3,0,6,0,...,3,4,2,2,0,13,0,11,1,3
2,1,0,11,0,0,10,1,0,0,4,...,0,2,0,0,2,8,1,13,0,4
3,0,9,0,3,0,1,3,0,4,0,...,48,11,2,0,0,4,0,2,0,0


In [7]:
test = pd.read_csv('./data/contest1/test.csv', index_col=0)
test[:3]

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
id,,,,,,,,,,,,,,,,,,,,,
1,4,2,2,0,4,2,1,2,1,0,...,4,5,3,1,0,7,1,6,2,3
2,3,3,0,0,1,2,0,9,3,3,...,8,1,0,0,1,21,0,27,2,0
3,0,0,0,2,2,8,0,1,0,9,...,3,23,0,0,0,1,0,5,1,1


In [11]:
X = train.drop(['label'], axis=1)
y = train['label']
X_kaggle = test

In [61]:
cross_val = StratifiedKFold(y, n_folds=5, random_state=42)

In [112]:
bsts = []
alpha = 0.2
train_predictions = np.zeros_like(y)
test_predictions = []
for itr, ite in tqdm(cross_val):    
    X_train, y_train, X_test, y_test = X.iloc[itr], y.iloc[itr], X.iloc[ite], y.iloc[ite]
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    bst_sgd = SGDClassifier(loss='log', n_iter=80, n_jobs=-1)
    bst_logreg = LogisticRegression()
    bst_sgd.fit(X_train, y_train)
    bst_logreg.fit(X_train, y_train)
    print(roc_auc_score(y_test, bst_sgd.predict(X_test) ))
    print(roc_auc_score(y_test, bst_logreg.predict(X_test) ))
    
    y_pred = np.argmax(alpha * bst_sgd.predict_proba(X_test) + (1 - alpha) * bst_logreg.predict_proba(X_test), axis=1)
    print(roc_auc_score(y_test, y_pred))
    
    
    bsts.append(bst_sgd)
    train_predictions[ite] = y_pred
    X_kaggle_scaled = scaler.transform(X_kaggle)
    y_test_pred = np.argmax(alpha * bst_sgd.predict_proba(X_kaggle_scaled) + (1 - alpha) * bst_logreg.predict_proba(X_kaggle_scaled), 
                            axis=1)
    test_predictions.append(y_test_pred)
    
print('ALL TRAIN SCORE:\n{}'.format(roc_auc_score(train_predictions, y)))

  0%|          | 0/5 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
 20%|██        | 1/5 [00:00<00:00,  8.93it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
 40%|████      | 2/5 [00:00<00:00,  8.29it/s]

0.8288537549407116
0.8861001317523057
0.860144927536232
0.9057312252964426
0.8894598155467721
0.9057312252964426


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
 60%|██████    | 3/5 [00:00<00:00,  8.00it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
 80%|████████  | 4/5 [00:00<00:00,  7.93it/s]

0.9234113712374582
0.9050167224080268
0.9137123745819399
0.8451417004048583
0.839676113360324
0.8484480431848852


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
100%|██████████| 5/5 [00:00<00:00,  7.90it/s]

0.8846828609986505
0.8836032388663968
0.8946018893387314
ALL TRAIN SCORE:
0.8830143540669856


In [113]:
y_pred_kaggle = [1 if x > 2.5 else 0 for x in np.sum(test_predictions, axis=0)]

In [114]:
ss = pd.read_csv('./data/contest1/sampleSubmission.csv')

In [115]:
ss.index = ss['id']
ss = ss.drop('id', axis=1)
ss.label = y_pred_kaggle

In [116]:
ss.to_csv('./contest1_expected_89')